## 28.3.2021 ##
### First draft of abstract ###
In our first meeting with the group, we discussed about the topic and created a first draft of our abstract.
For this, everyone wrote a draft and we combined our results in the end.

## 29.3.2021 ##
### Revision of abstract ###
In order to comply with the structure shown in https://www.emerald.com/insight/content/doi/10.1108/JD-02-2020-0030/full/html, I revised and restructured our abstract and uploaded the new version the next day, after the other group members agreed with the text.

## 3.4.2021 ##
### First draft of Data Management Plan (DMP) ###
In a group meeting, we created a DMP on the platform Argos (https://argos.openaire.eu/) and divided the further work on it (the description of the two datasets) between us.

## 4.4.2021 ##
### Adding small parts to DMP ###
I concentrated on the parts 6 and 7 of the datasets. When we were all done with our part of the work, we created the first version of our DMP: https://doi.org/10.5281/zenodo.4663016

## 6.4.2021 ##
### Revising first version of DMP ###
Reading the first version of our DMP, I noticed some issues that we revised and updated, creating this version: https://doi.org/10.5281/zenodo.4665853

## 10.4.2021 ##
### Literature review ###
Based on a Google Scholar search with the keywords "wrong doi" "invalid doi" and "doi error", I started with the literature review for the project. I continued the search following the cited references that seemed to contain information about error classes occuring in DOI names and (automatic) methods for fixing and cleaning the DOIs. Especially Xu, S., Hao, L., An, X. et al. Types of DOI errors of cited references in Web of Science with a cleaning method. Scientometrics 120, 1427–1437 (2019). https://doi.org/10.1007/s11192-019-03162-4 (which we already found before) seems to be useful.

## 11.04.2021 ##
### First draft of workflow in group ###
Based on the literature review, we created a workflow very similar to the one described by Xu et al. using https://www.protocols.io/ .\
Classes of errors: prefix-type, suffix-type and other-type errors\
Main method used to clean errors: regular expressions\
Input: CSV file https://zenodo.org/record/4625300/files/invalid_dois.csv\
Output: CSV file: cited DOI, modified DOI, type of error\
Before and after procedure: check validity of DOI making use of the DOI API

## 16.04.2021 ##
### Trying to implement DOI check ###
Code to check one DOI by replacing handle var:

In [103]:
import requests as req
handle = "10.1016/j.eswa.2007.09.055"
r = req.get("https://doi.org/api/handles/" + handle)
print(r.json())

{'responseCode': 1, 'handle': '10.1016/j.eswa.2007.09.055', 'values': [{'index': 1, 'type': 'URL', 'data': {'format': 'string', 'value': 'https://linkinghub.elsevier.com/retrieve/pii/S0957417407004629'}, 'ttl': 86400, 'timestamp': '2019-01-03T12:20:32Z'}, {'index': 700050, 'type': '700050', 'data': {'format': 'string', 'value': '20111118023200000'}, 'ttl': 86400, 'timestamp': '2019-01-03T12:20:32Z'}, {'index': 100, 'type': 'HS_ADMIN', 'data': {'format': 'admin', 'value': {'handle': '0.na/10.1016', 'index': 200, 'permissions': '111111110010'}}, 'ttl': 86400, 'timestamp': '2019-01-03T12:20:32Z'}]}


Code used for first attempt to check whole dataset:

In [ ]:
import requests as req
import csv

with open("invalid_dois.csv", encoding='Latin1') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    with open("doi_check.csv", "w", newline='', encoding='Latin1') as result:
        csv_writer = csv.writer(result, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(['DOI', 'ResponseCode'])
        for row in csv_reader:
            if row[0] != "Valid_citing_DOI":
                r = req.get("https://doi.org/api/handles/" + row[1])
                rjson = r.json()
                csv_writer.writerow([rjson["handle"], str(rjson["responseCode"])])
                if rjson["responseCode"] != 100:
                    print(rjson["handle"] + ",Code " + str(rjson["responseCode"]))


In [17]:
import csv

with open("invalid_dois.csv", encoding='Latin1') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    print(len(list(csv_reader)))

1223297


Trying to check the results, the code ran for about one hour before stopping with a connection error. In this time, 5707 results were created. Like this, for the whole dataset, we would need 214 hours to compute, which is quite unrealistic.

## 18.04.2021 ##
### Trying to improve code, investigating output from first attempt ###
Proposed code for future attempts, leaving out all valid DOIs:

In [ ]:
import requests as req
import csv

with open("invalid_dois.csv", encoding='Latin1') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    with open("only_invalid.csv", "w", newline='', encoding='Latin1') as result:
        csv_writer = csv.writer(result, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(['DOI', 'ResponseCode'])
        for row in csv_reader:
            if row[0] != "Valid_citing_DOI":
                r = req.get("https://doi.org/api/handles/" + row[1])
                rjson = r.json()
                if rjson["responseCode"] != 1:
                    csv_writer.writerow([rjson["handle"], str(rjson["responseCode"])])
                if rjson["responseCode"] != 100:
                    print(rjson["handle"] + ",Code " + str(rjson["responseCode"]))

#TODO: add possibility to restart where we left after crash?
#TODO: correct output format (leave out valid dois? more columns?)
#TODO: how make it possible to compute all?

Investigating on the result list from 16.4.2021, I found out about some errors and solutions:

Erroneous comma in DOI:
- Replace comma with dot, e.g. 10.1016/j.eswa,2007.09.055 --> 10.1016/j.eswa.2007.09.055
- Leave out comma, e.g. 10.1038/s,41598-017-04125-6 --> 10.1038/s41598-017-04125-6

Erroneous strings added to end of DOI:
- Leave out part after comma, e.g. 10.1002/asi.22647,pp.2206-2222 --> 10.1002/asi.22647
- Leave out part after many dots, e.g. 10.1021/cm980610m........11,23(1999) --> 10.1021/cm980610m
- Leave out whitespaces (after comma?), e.g. after doi: 10.1016/j.bar.2014.06.001
- Leave out dot at end of doi, e.g. 10.1038/bdjopen.2017.10. --> 10.1038/bdjopen.2017.10
- Leave out "/pdf" at end: e.g. 10.1088/1755-1315/95/5/052007/pdf --> 10.1088/1755-1315/95/5/052007

Others:
- Missing suffix, e.g. 10.1016/
- 301 response: prefix error? e.g. for doi: 10.16/j.amjsurg.2005.10.023